In [1]:

from MVPAnalyzer import Analyzer
import numpy as np
import matplotlib.pyplot as plt

# mvpa = Analyzer('/home/maxw/Documents/ESSTECH25/MVP300_DATA/Stationary_Profiling/',subdirs=True)
mvpa = Analyzer('/home/maxw/Documents/ESSTECH25/MVP300_DATA/Underway profile/A to B - 4 Knots/')

# mvpa.help()

# mvpa.load_mvp_data() 
mvpa.load_mvp_data(delp=[8,9,10,11,18,19]) # manually delete profiles because the MVP was stopped and restarted during the cast

# mvpa.load_ctd_data('C:/Users/maxim/Documents/INGE CNRS/ESSTECH25/BATHYSONDE/DATA/TRAIT/CNV/')
# mvpa.keep_selected_profiles([0, 10, 8, 2], [0, 2, 4, 8])

mvpa.keep_selected_profiles([2,4,6])

mvpa.print_profile_metadata()

mvpa.viscous_heating_correction()
print('Viscous heating correction done.')
mvpa.filtering_surface_waves()
print('Surface wave filtering done.')
mvpa.temporal_lag_correction(correction=True)
print('Temporal lag correction done.')

/home/maxw/miniconda3/envs/oceanapp/lib/python3.11/site-packages/seabird/cnv.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


/home/maxw/miniconda3/envs/oceanapp/lib/python3.11/site-packages/seabird/cnv.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Found 24 MVP files in the directory: /home/maxw/Documents/ESSTECH25/MVP300_DATA/Underway profile/A to B - 4 Knots/
MVP data loaded successfully.
MVP data:
Number of profiles: 3
  Profil down 0 - Profil up 1 - Latitude: 42.75363, Longitude: 6.17481, Date/Heure: 2025-08-09 13:42:26
  Profil down 2 - Profil up 3 - Latitude: 42.72784, Longitude: 6.16614, Date/Heure: 2025-08-09 14:08:17
  Profil down 4 - Profil up 5 - Latitude: 42.70016, Longitude: 6.15697, Date/Heure: 2025-08-09 14:35:10
Viscous heating correction done.
Surface wave filtering done.
MVP data loaded successfully.
MVP data:
Number of profiles: 3
  Profil down 0 - Profil up 1 - Latitude: 42.75363, Longitude: 6.17481, Date/Heure: 2025-08-09 13:42:26
  Profil down 2 - Profil up 3 - Latitude: 42.72784, Longitude: 6.16614, Date/Heure: 2025-08-09 14:08:17
  Profil down 4 - Profil up 5 - Latitude: 42.70016, Longitude: 6.15697, Date/Heure: 2025-08-09 14:35:10
Viscous heating correction done.
Surface wave filtering done.
Temporal lag 

In [49]:
import findThermalLagParams
import importlib
importlib.reload(findThermalLagParams)
from findThermalLagParams import find_thermal_lag_params

id = 2

time1 = mvpa.TIME0[id]
time2 = mvpa.TIME0[id+1]
cond1 = mvpa.COND_align[id]
cond2 = mvpa.COND_align[id+1]
temp1 = mvpa.TEMP_align[id]
temp2 = mvpa.TEMP_align[id+1]
pres1 = mvpa.PRES0[id]    
pres2 = mvpa.PRES0[id+1]



# Calcul robuste de la vitesse avec données nettoyées mais structure préservée
fs_fixed = 25.0  # Hz - fréquence d'échantillonnage typique des MVP

# Masques pour identifier les données valides
mask1 = ~np.isnan(time1) & ~np.isnan(pres1)
mask2 = ~np.isnan(time2) & ~np.isnan(pres2)

# Calcul des vitesses verticales avec structure préservée
v_z1 = np.full_like(pres1, np.nan)  # Initialiser avec NaN
v_z2 = np.full_like(pres2, np.nan)

# Calculer gradient seulement pour les données valides
if np.sum(mask1) > 1:
    pres1_valid = pres1[mask1]
    v_z1_valid = np.gradient(pres1_valid) * fs_fixed  # dbar/s * Hz -> m/s approx
    v_z1[mask1] = v_z1_valid

if np.sum(mask2) > 1:
    pres2_valid = pres2[mask2]
    v_z2_valid = np.gradient(pres2_valid) * fs_fixed
    v_z2[mask2] = v_z2_valid

# Lissage avec gestion des NaN
window = 5
v_z1_smooth = np.full_like(v_z1, np.nan)
v_z2_smooth = np.full_like(v_z2, np.nan)

if np.sum(~np.isnan(v_z1)) > 2*window:
    # Appliquer convolution seulement aux données valides
    valid_indices1 = np.where(~np.isnan(v_z1))[0]
    if len(valid_indices1) > 2*window:
        v_z1_temp = np.convolve(v_z1[~np.isnan(v_z1)], np.ones(2*window+1)/(2*window+1), mode='same')
        v_z1_smooth[valid_indices1] = v_z1_temp

if np.sum(~np.isnan(v_z2)) > 2*window:
    valid_indices2 = np.where(~np.isnan(v_z2))[0]
    if len(valid_indices2) > 2*window:
        v_z2_temp = np.convolve(v_z2[~np.isnan(v_z2)], np.ones(2*window+1)/(2*window+1), mode='same')
        v_z2_smooth[valid_indices2] = v_z2_temp

# Utiliser données lissées si disponibles, sinon données brutes
v_z1 = np.where(~np.isnan(v_z1_smooth), v_z1_smooth, v_z1)
v_z2 = np.where(~np.isnan(v_z2_smooth), v_z2_smooth, v_z2)

v_z1 = np.abs(v_z1)
v_z2 = np.abs(v_z2)

v_hrz = 4*0.5144 # 4 knots in m/s

# Calcul du flux total avec gestion des NaN
flow1 = np.full_like(v_z1, v_hrz)  # Initialiser avec vitesse horizontale minimum
flow2 = np.full_like(v_z2, v_hrz)

# Ajouter composante verticale quand disponible
valid_v_z1 = ~np.isnan(v_z1)
valid_v_z2 = ~np.isnan(v_z2)

flow1[valid_v_z1] = np.sqrt(v_hrz**2 + v_z1[valid_v_z1]**2)
flow2[valid_v_z2] = np.sqrt(v_hrz**2 + v_z2[valid_v_z2]**2)



# CTD non pompé (flux variable)
params, exitflag, residual = find_thermal_lag_params(
    time1, cond1, temp1, pres1,
    time2, cond2, temp2, pres2,
    flow1=flow1, flow2=flow2,
    graphics=False
)

15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 395 23739
15761 39

In [24]:
%matplotlib tk

In [53]:
from findThermalLagParams import correct_thermal_lag
import gsw
temp_corr,cond_corr = correct_thermal_lag(time1, cond1, temp1, params, flow1)

SAL_RAW1 = gsw.SP_from_C(cond1, temp1, pres1)
SAL_RAW2 = gsw.SP_from_C(cond2, temp2, pres2)
sal_corr1 = gsw.SP_from_C(cond1, temp_corr, pres1)
sal_corr2 = gsw.SP_from_C(cond_corr, temp1, pres1)


print(temp_corr)


plt.figure()
plt.plot(temp1, pres1, label='Temp 1 originale')
plt.plot(temp2, pres2, label='Temp 2 originale')
plt.plot(temp_corr, pres1, label='Temp 1 corrigée')
plt.gca().invert_yaxis()
plt.xlabel('Température (°C)')
plt.ylabel('Pression (dbar)')
plt.legend()
plt.title(f'Correction thermique avec params: {params}')
plt.show()

plt.figure()
plt.plot(cond1, pres1, label='Cond 1 originale')
plt.plot(cond2, pres2, label='Cond 2 originale')
plt.plot(cond_corr, pres1, label='Cond 1 corrigée')
plt.gca().invert_yaxis()
plt.xlabel('Conductivité (mS/cm)')
plt.ylabel('Pression (dbar)')
plt.legend()
plt.title('Conductivité corrigée vs originale')
plt.show()

plt.figure()
plt.plot(sal_corr1, pres1, label='Sal 1 corrigée')
plt.plot(sal_corr2, pres1, label='Sal 2 corrigée')
plt.plot(SAL_RAW1, pres1, label='Sal 1 originale', alpha=0.5)
plt.plot(SAL_RAW2, pres2, label='Sal 2 originale', alpha=0.5)
plt.gca().invert_yaxis()
plt.xlabel('Salinité (PSU)')
plt.ylabel('Pression (dbar)')
plt.legend()
plt.title('Salinité corrigée vs originale')
plt.show()

[nan nan nan ... nan nan nan]
